In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import scanpy as sc

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [44]:
import mushroom.data.cosmx as cosmx
import mushroom.data.multiplex as multiplex
import mushroom.registration.bigwarp as bigwarp
import mushroom.utils as utils

## HT704B1

In [1]:
# visium hd
# /diskmnt/primary/Spatial_Transcriptomics/VisiumHD_run/FFPE/SpacerangerOut/HTAN/BRCA/HT704B1-S1H3Fp1//HT704B1-S1H3Fp1U2/outs
# /diskmnt/primary/Spatial_Transcriptomics/VisiumHD_run/FFPE/SpacerangerOut/HTAN/BRCA/HT704B1-S1H3Fp1//HT704B1-S1H3Fp1U51/outs

# he
# /diskmnt/Projects/Users/estorrs/imaging-analysis/data/htan_talk/visium_hd/HT704B1-S1H3Fp1U2.ome.tif
# /diskmnt/Projects/Users/estorrs/imaging-analysis/data/htan_talk/visium_hd/HT704B1-S1H3Fp1U51.ome.tif

In [ ]:







visium_dir = Path('/diskmnt/primary/Spatial_Transcriptomics/VisiumHD_run/FFPE/SpacerangerOut/HTAN/BRCA/HT704B1-S1H3Fp1//HT704B1-S1H3Fp1U2/outs')
he_fp = ''
